In [2]:
import requests
import json

# --- 1. Define the Location Function ---
def test_location_module(city, specialty=None):
    """
    Finds medical services using Free OpenStreetMap API (Nominatim)
    and generates Google Maps navigation links.
    """
    print(f"🔎 Searching for '{specialty if specialty else 'Hospitals'}' in {city}...")
    
    # Base URL for the free API
    url = "https://nominatim.openstreetmap.org/search"
    
    # Search Query Construction
    if specialty:
        search_query = f"{specialty} in {city}"
    else:
        search_query = f"Hospitals in {city}"
        
    # Parameters (format=json is crucial)
    params = {
        'q': search_query,
        'format': 'json',
        'addressdetails': 1,
        'limit': 5,  # Let's get top 3 results for testing
        'extratags': 1
    }
    
    # Headers (REQUIRED by Nominatim policy to identify your app)
    headers = {
        'User-Agent': 'StudentMedicalProject/1.0' 
    }
    
    try:
        # Make the request
        response = requests.get(url, params=params, headers=headers)
        
        # Check if successful
        if response.status_code != 200:
            return f"Error: API returned status {response.status_code}"
            
        results = response.json()
        
        if not results:
            return "❌ No results found. Try a bigger city name."
            
        # Format and Print Results
        print(f"\n✅ Found {len(results)} locations:\n")
        
        for idx, place in enumerate(results, 1):
            name = place.get('display_name', 'Unknown').split(',')[0]
            extra = place.get('extratags', {})
            phone = extra.get('phone') or extra.get('contact:phone') or "Not Listed"
            website = extra.get('website') or extra.get('contact:website') or "Not Listed"
            lat = place.get('lat')
            lon = place.get('lon')
            
            # TRICK: Use OSM data to build a Google Maps Link
            google_maps_link = f"https://www.google.com/maps/search/?api=1&query={lat},{lon}"
            
            print(f"{idx}. 🏥 {name}")
            print(f"   📞 Phone: {phone}")
            print(f"   🌐 Web:   {website}")
            print(f"   📍 Location: {lat}, {lon}")
            print(f"   🔗 Map Link: {google_maps_link}")
            print("   " + "-"*40)
            
    except Exception as e:
        return f"System Error: {e}"

# --- 2. Run Test Cases ---

# Test Case A: General Hospital Search
print("--- TEST CASE A: General Search ---")
test_location_module("Paramakudi") 

print("\n")

# Test Case B: Specialist Search
print("--- TEST CASE B: Specialist Search ---")
test_location_module("Coimbatore", "Cardiologist")

print("\n")

# Test Case C: Invalid/Empty Search (To see how it handles errors)
print("--- TEST CASE C: Fake City ---")
test_location_module("AtlantisUnderSea")

--- TEST CASE A: General Search ---
🔎 Searching for 'Hospitals' in Paramakudi...

✅ Found 5 locations:

1. 🏥 Ganeshamoorthi Hospital
   📞 Phone: Not Listed
   🌐 Web:   Not Listed
   📍 Location: 9.5409777, 78.5957703
   🔗 Map Link: https://www.google.com/maps/search/?api=1&query=9.5409777,78.5957703
   ----------------------------------------
2. 🏥 Krishna Moorthy Hospital
   📞 Phone: Not Listed
   🌐 Web:   Not Listed
   📍 Location: 9.5423103, 78.5962770
   🔗 Map Link: https://www.google.com/maps/search/?api=1&query=9.5423103,78.5962770
   ----------------------------------------
3. 🏥 Balavinayagar Hospital
   📞 Phone: Not Listed
   🌐 Web:   Not Listed
   📍 Location: 9.5404973, 78.5946543
   🔗 Map Link: https://www.google.com/maps/search/?api=1&query=9.5404973,78.5946543
   ----------------------------------------
4. 🏥 Dr. Vengatachalapathy Hospital
   📞 Phone: Not Listed
   🌐 Web:   Not Listed
   📍 Location: 9.5416308, 78.5954023
   🔗 Map Link: https://www.google.com/maps/search/?api=1&

'❌ No results found. Try a bigger city name.'

In [1]:
import requests
import json

def find_medical_services_robust(city, specialty=None):
    """
    Robust search that tries specific specialist first, then falls back 
    to general hospitals if nothing is found. Also fetches Contact Info.
    """
    
    # 1. Setup API parameters
    base_url = "https://nominatim.openstreetmap.org/search"
    headers = { 'User-Agent': 'SmartMedicalAssistant_StudentProject/1.0' }

    def _get_tag(tags, keys):
        if not tags: return None
        for key in keys:
            if tags.get(key):
                return tags.get(key)
        return "Not Listed"
    
    def run_query(query_text):
        params = {
            'q': query_text,
            'format': 'json',
            'addressdetails': 1,
            'limit': 5,
            'extratags': 1  # Asks for phone/website data
        }
        return requests.get(base_url, params=params, headers=headers).json()

    # 2. Strategy A: Try Specific Search
    search_term = f"{specialty} in {city}" if specialty else f"Hospitals in {city}"
    print(f"🔎 Strategy A: Searching for '{search_term}'...")
    
    results = run_query(search_term)

    # 3. Strategy B: Fallback
    if not results and specialty:
        print(f"⚠️ No specific '{specialty}' found. Switching to Strategy B (General Hospitals)...")
        fallback_query = f"Hospitals in {city}"
        results = run_query(fallback_query)
        if results:
            print(f"✅ Found general hospitals instead.")

    if not results:
        print(f"❌ No medical centers found in {city}. Check spelling.")
        return

    # 4. Format Output with Contacts
    print(f"\n🏥 --- Results for {city} ---")
    
    for idx, place in enumerate(results, 1):
        name = place.get('display_name', 'Unknown').split(',')[0]
        
        # --- FIX IS HERE ---
        # We use 'or {}' to ensure 'extra' is always a dictionary, 
        # even if the API sends None (null).
        extra = place.get('extratags') or {}
        
        # phone = extra.get('phone') or extra.get('contact:phone') or "Not Listed"
        # website = extra.get('website') or extra.get('contact:website') or "Not Listed"

        phone = _get_tag(extra, ['phone', 'contact:phone', 'contact:mobile', 'contact:landline'])
            
        # Check all possible keys for website
        website = _get_tag(extra, ['website', 'contact:website', 'url'])
        
        # Opening Hours (Bonus)
        hours = _get_tag(extra, ['opening_hours'])
        
        # Create Google Maps Link
        lat, lon = place.get('lat'), place.get('lon')
        map_link = f"https://www.google.com/maps/search/?api=1&query={lat},{lon}"
        
        print(f"{idx}. **{name}**")
        print(f"   📞 Phone: {phone}")
        print(f"   🌐 Web:   {website}")
        print(f"   📍 Map:   {map_link}")
        print(f"   ⏱️ Open:  {hours} ")
        print("   " + "-"*40)


--- TEST 1: General ---
🔎 Strategy A: Searching for 'Hospitals in Paramakudi'...

🏥 --- Results for Paramakudi ---
1. **Ganeshamoorthi Hospital**
   📞 Phone: Not Listed
   🌐 Web:   Not Listed
   📍 Map:   https://www.google.com/maps/search/?api=1&query=9.5409777,78.5957703
   ⏱️ Open:  Not Listed 
   ----------------------------------------
2. **Krishna Moorthy Hospital**
   📞 Phone: Not Listed
   🌐 Web:   Not Listed
   📍 Map:   https://www.google.com/maps/search/?api=1&query=9.5423103,78.5962770
   ⏱️ Open:  Not Listed 
   ----------------------------------------
3. **Balavinayagar Hospital**
   📞 Phone: Not Listed
   🌐 Web:   Not Listed
   📍 Map:   https://www.google.com/maps/search/?api=1&query=9.5404973,78.5946543
   ⏱️ Open:  Not Listed 
   ----------------------------------------
4. **Dr. Vengatachalapathy Hospital**
   📞 Phone: Not Listed
   🌐 Web:   Not Listed
   📍 Map:   https://www.google.com/maps/search/?api=1&query=9.5416308,78.5954023
   ⏱️ Open:  Not Listed 
   -----------

In [ ]:

# --- TEST AREA ---

# Test 1: General Search
print("--- TEST 1: General ---")
find_medical_services_robust("Paramakudi")

print("\n")

# Test 2: Specialist Search
print("--- TEST 2: Specialist ---")
find_medical_services_robust("Coimbatore", "Cardiologist")